# todo: -itd分开的作用

    -i 让容器的标准输入保持打开, 以交互模式运行容器，通常与 -t 同时使用；

In [ ]:
启动容器 -i
$  docker run -i ubuntu:18.04

查看容器启用状态
$  docker container ls -a
Up 4 seconds    

退出容器
$ exit

再次查看容器启用状态
$ docker container ls -a
Exited (0) 3 seconds ago    


    -t 选项让Docker分配一个伪终端（pseudo-tty）并绑定到容器的标准输入上,  通常与 -i 同时使用；

In [ ]:
启动容器 -t
$ docker run -t ubuntu:18.04 /bin/bash

root@bea30c9436a6:/# ls   伪终端 没有输出

查看容器启用状态
$  docker container ls -a
Up 4 seconds  

此时伪终端无法退出，可以另起一个terminal, 输入一下指令
$ docker container stop [容器ID]

再次查看容器启用状态
$ docker container ls -a
Exited (0) 3 seconds ago  

    -d: 后台运行容器，并返回容器ID；

In [ ]:
未使用 -d 参数

直接把执行命令的结果输出在当前宿主机下
$ docker run ubuntu:18.04 /bin/sh -c "while true; do echo hello world; sleep 1; done"
    
容器会把输出的结果 (STDOUT) 打印到宿主机上面
hello world
hello world
hello world
.......

查看容器启用状态
$  docker container ls -a
Up 4 seconds    

退出 crtl+c

再次查看容器启用状态
$ docker container ls -a
Exited (0) 3 seconds ago 


In [ ]:
如果使用了 -d 参数运行容器。
$ docker run -d ubuntu:18.04 /bin/sh -c "while true; do echo hello world; sleep 1; done"

查看容器启用状态
$  docker container ls -a
Up 4 seconds  

此时容器会在后台运行并不会把输出的结果 (STDOUT) 打印到宿主机上面(输出结果可以用 docker logs 查看)。

$ docker container logs [container ID or NAMES]
hello world
hello world
hello world
. . .

终止容器，停掉上面后台运行的容器
$ docker container stop 93f70ee34376

In [ ]:
#清理所有终止状态的容器
$ docker container prune

# Untagged 和 Deleted  （镜像删除）

    1、除了镜像依赖以外，还需要注意的是容器对镜像的依赖。如果有用这个镜像启动的容器存在（即使容器没有运行），那么同样不可以删除这个镜像

In [ ]:
切换到 
$ cd /Users/xuwenxiang/Documents/docker_share/rm_image
在这个目标下有3个子文件，分别进入子文件并构建3个子镜像

构建镜像
$ cd test1
$ docker build -t rm_image:v1 .
$ cd test2
$ docker build -t rm_image:v2 .
$ cd test3
$ docker build -t rm_image:v3 .

查看构建的镜像
$docker image ls
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
rm_image            v1                  4e5021d210f6        4 weeks ago         64.2MB
rm_image            v2                  4e5021d210f6        4 weeks ago         64.2MB
rm_image            v3                  4e5021d210f6        4 weeks ago         64.2MB
ubuntu              18.04               4e5021d210f6        4 weeks ago         64.2MB

启用rm_image:v3镜像
$ docker run -it rm_image:v3

准备删除 rm_image:v3
$ docker image rm rm_image:v3
删除不了，提示镜像整备某个容器使用

停止容器
$ exit
继续试着删除rm_image:v3
$ docker image rm rm_image:v3
发现还是无法删除
    

In [ ]:
删除rm_image:v3对应的容器
$ docker container rm [容器id]

继续试着删除rm_image:v3
$ docker image rm rm_image:v3

Untagged: rm_image:v3
Deleted: sha256:5377db43d6e7aca3539870ad11e65ba53c26be04b8c8a8d3c022e9c382bf4fd7
Deleted: sha256:05790d44282e906d7a86b4ff7f194f6d6c23b3a48f6bb1db325786c8af32c42a
        
这个时候我们可以看到镜像被删除了，


     2、 因此当我们使用上面命令删除镜像的时候，实际上是在要求删除某个标签的镜像。所以首先需要做的是将满足我们要求的所有镜像标签都取消，这就是我们看到的 Untagged 的信息。因为一个镜像可以对应多个标签，因此当我们删除了所指定的标签后，可能还有别的标签指向了这个镜像，如果是这种情况，那么 Delete 行为就不会发生。所以并非所有的docker image rm 都会产生删除镜像的行为，有可能仅仅是取消了某个标签而已。

In [ ]:
接着我们扎test2文件夹构建2个copy镜像
# 
$ cd /Users/xuwenxiang/Documents/docker_share/rm_image/test2
$ docker build -t rm_image:v2_copy .
$ docker build -t rm_image:v2_copy2 .

查看镜像
$docker image ls
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
rm_image            v2                  a6f3bfb93acd        11 minutes ago      64.2MB
rm_image            v2_copy             a6f3bfb93acd        11 minutes ago      64.2MB
rm_image            v2_copy2            a6f3bfb93acd        11 minutes ago      64.2MB

这时候rm_image_v2 有3个标签分别是v2， v2_copy， v2_copy2

试着删除标签为v2_copy2的镜像，
$ docker image rm rm_image:v2_copy2
Untagged: rm_image:v2_copy2

查看删除情况
$docker image ls
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
rm_image            v2                  a6f3bfb93acd        11 minutes ago      64.2MB
rm_image            v2_copy             a6f3bfb93acd        11 minutes ago      64.2MB

可以发现只触发了Untagged， 只是将标签取消了， 既将标签为v2_copy2的镜像删除了

继续删:rm_image：v2_copy
$ docker image rm rm_image:v2_copy
 Untagged: rm_image:v2_copy
 结果和v2_copy2一样

继续删除rm_image:v2
$ docker image rm rm_image:v2
 Untagged: rm_image:v2
 Deleted: sha256:a6f3bfb93acd58794e65f5aff8aaef460df697650c03daac5c0e7d627af7d54c
 Deleted: sha256:ab99c4e7572c2800d5a7a2517700cafac11fc309f7a69c30d66e961c9c6e6ba6   
 所有的标签都删除后，才会看到Deleted行为




<!--     因此很有可能某个其它镜像正依赖于当前镜像的某一层。这种情况，依旧不会触发删除该层的行为。直到没有任何层依赖当前层时，才会真实的删除当前层。 -->
<!-- # 切换到test4文件夹
# 构建test4
# $docker build -t rm_image:v4 .

# 启动容器
# $ docker run -it rm_image:v4
# 增加一个文件
# $ echo '<h1>Hello, Docker!</h1>' > index2.html

# 查看刚才的容器名称
# $ docker container ls -a

# docker commit在rm_image:v4基础上构建新镜像
# $ docker commit --author "synyi123 <970784198@qq.com>" --message 'test' 43d7ad18f6e7 rm_image:v4_update
    
# 删除 rm_image:v4 -->

# cmd 具体场景

    CMD 和 ENTRYPOINT 指令都是用来指定容器启动时运行的命令。
    都有exec 模式和 shell 模式，功能类似

    有一种场景
    由 CMD 指令指定ENTRYPOINT默认的可选参数

In [ ]:
切换到 /Users/xuwenxiang/Documents/docker_share/entrypoint_2
$ cd /Users/xuwenxiang/Documents/docker_share/entrypoint_2

$ docker build -t myip .  # 这一步较慢，可以查看dockerfile， 直接启动镜像
$ docker run myip

HTTP/1.1 200 OK
Date: Thu, 23 Apr 2020 09:23:51 GMT
Content-Type: text/html
Server: Kestrel
Content-Length: 2059
Last-Modified: Mon, 17 Feb 2020 05:16:45 GMT
Accept-Ranges: bytes
ETag: "1d5e551724c148b"

    注：
    Dockerfile 中至少要有一个

# EXPOSE 声明端口

In [ ]:
-P: 随机端口映射，容器内部端口随机映射到主机的高端口
-p: 指定端口映射，格式为：主机(宿主)端口:容器端口

In [ ]:
使用镜像nginx:latest以后台模式启动一个容器

将容器的80端口映射到主机的80端口
$ docker run --name webserver -d -p 80:80 nginx
    
将容器的80端口映射到主机随机端口
$ docker run -P -d nginx:latest

查看容器运行状态, 可以查看到刚才随机映射的主机端口未32768
$ docker container ls -a
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                   NAMES
6aeed76c0807        nginx:latest        "nginx -g 'daemon of…"   51 seconds ago      Up 48 seconds       0.0.0.0:32768->80/tcp   sad_cray   

In [ ]:
dockerfile 文件：

In [ ]:
FROM nginx:latest

EXPOSE 801

In [ ]:
切换到 /Users/xuwenxiang/Documents/docker_share/expose
$ cd /Users/xuwenxiang/Documents/docker_share/expose

构建镜像
$ docker build -t expose .

启动容器, -P 自动映射(宿主机端口随机)
$ docker run --name expose_test -P -d expose

$ docker container ls -a
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
5b5a3c331769        expose              "nginx -g 'daemon of…"   4 seconds ago       Up 3 seconds        80/tcp, 801/tcp     expose_test

    解释：
     EXPOSE 仅仅是暴露一个端口，一个标识，在没有定义任何端口映射时，外部是无法访问到容器提供的服务。

# WORKDIR 指定工作目录, 多行run，WORKDIR是不是全局的

In [ ]:
# 切换到 /Users/xuwenxiang/Documents/docker_share/workdir
$ cd /Users/xuwenxiang/Documents/docker_share/workdir

In [ ]:
dockerfile文件内容如下

FROM ubuntu:18.04
WORKDIR /var/model
RUN echo '<h1>Hello, Docker!</h1>' > index.html
RUN echo '<h1>Nice to meet you!</h1>' > index2.html

In [ ]:
构建镜像
$ docker build -t workdir_test .

插看镜像
$ docker image ls

启动容器
$ docker run --name workdir_test -it workdir_test

可以看到两个文件都存在
root@a3a88f5ea147:/var/model# ls
index.html  index_2.html


     注：
     WORKDIR 指令可以来指定工作目录（或者称为当前目录），以后各层的当前目录就被改为指定的目录，如该目录不存在，WORKDIR 会帮你建立目录。

# volume挂载 

In [ ]:
# 切换到/Users/xuwenxiang/Documents/docker_share/volume目录下

    1、不使用挂载， 切换到test1

In [ ]:
构建镜像
$ cd test1 
$ docker build -t volume_test1 .

启用镜像
$ docker run -it volume_test1 /bin/bash

写入一个文件
$echo '<h1>Hello, Docker!</h1>' > index2.html
退出
$ exit 

再次启用镜像
$ docker run -it volume_test1 /bin/bash
$ ls 查看目录，发现刚才存的index2.html文件不在的。


 # 如何保证镜像重启时，所存储的文件能够保存下来

    2、使用挂载， 切换到test2,
    添加 VOLUME ["/data"]

In [ ]:
构建镜像
$ cd test2
$ docker build -t volume_test2 .

启用镜像
$ docker run -it volume_test2 /bin/bash

写入一个文件 docke.txt
$ echo '<h1>Hello, Docker!</h1>' > docker.html
#不要退出交互模式

查看挂载位置

$ docker inspect [container_id] | grep Source
"Source": "/var/lib/docker/volumes/90bf3beeb5363be912039acc519657bd1cebc7825920fe0e540b6f9442bff61d/_data"
这就是宿主机的位置

在宿主机查看， 另起一个terminal，输入一下指令
screen ~/Library/Containers/com.docker.docker/Data/vms/0/tty
$ cd /var/lib/docker/volumes/90bf3beeb5363be912039acc519657bd1cebc7825920fe0e540b6f9442bff61d/_data
$ ls 
docker.html

在挂载位置新建文件
$ echo '<h1>Hello, host!</h1>' > host.html
$ls 
docker.html  host.html

然后再docker容器内查看
$ls 
docker.html  host.html
发现在data文件夹有host.html

$exit 退出容器，我们发现文件还是存在宿主机上


In [ ]:
VOLUME ["/data"]， 由docker 随机生成挂载点，也可以手动指定挂载点，
启用镜像时 覆盖这个VOLUME ["/data"]

挂载点为：/Users/xuwenxiang/Documents/docker_share/share
$ docker run -it -v /Users/xuwenxiang/Documents/docker_share/share:/data volume_test2 /bin/bash
$ echo '<h1>Hello, Docker!</h1>' > docker.html

在宿主机/Users/xuwenxiang/Documents/docker_share/share查看
文件被保留下来了

在宿主机创建文件，在docker容器查看发现一样被保存下来了

$exit 退出容器，我们发现文件还是存在宿主机上

In [1]:
pwd

'/Users/xuwenxiang'